# SKTime
## Imports

In [1]:
import pandas as pd
from sktime.split import temporal_train_test_split

## Data 

In [2]:
df = pd.read_csv('datawarehouse/Merged.csv')
df.head(3)

,AID_YEAR,GROUP_ID,ACADEMIC_LEVEL_TERM_START,ACADEMIC_PLAN,ACADEMIC_PROGRAM_DESC,FIN_AID_FED_RES,UVA_ACCESS,REPORT_CODE,Need based,INST_NEED,OFFER_BALANCE,FUNDED_PARTY,TOTAL_PARTY
0,2015,6,Level Four,AERO-2MJ,Engineering,N,Federal - Grants,FE9,Total,286760.0,25932.0,1.0,1.0
1,2016,6,Level Four,AERO-2MJ,Engineering,N,Federal - Grants,FE9,Total,0.0,0.0,0.0,0.0
2,2017,6,Level Four,AERO-2MJ,Engineering,N,Federal - Grants,FE9,Total,0.0,0.0,0.0,0.0


In [3]:
targets = ['OFFER_BALANCE']
date = 'AID_YEAR'

hierarchies = [col for col in df.columns if df[col].dtype == 'object']
print(f'Hierarchies: {hierarchies}\nTarget {targets} Timepoint {date}.')
print(df[date].min(), df[date].max())

Hierarchies: ['ACADEMIC_LEVEL_TERM_START', 'ACADEMIC_PLAN', 'ACADEMIC_PROGRAM_DESC', 'FIN_AID_FED_RES', 'UVA_ACCESS', 'REPORT_CODE', 'Need based']
Target ['OFFER_BALANCE'] Timepoint AID_YEAR.
2015 2024


In [4]:
min_year = df[date].min()
df[date] -= min_year
df = df[hierarchies+[date]+targets]

In [5]:
# take first 10 time series groups
data = df.iloc[:(df[date].max()+1)*10]

In [6]:
data.set_index(hierarchies + [date], inplace=True)
data.head(3)

OFFER_BALANCE
ACADEMIC_LEVEL_TERM_START ACADEMIC_PLAN ACADEMIC_PROGRAM_DESC FIN_AID_FED_RES UVA_ACCESS       REPORT_CODE Need based AID_YEAR               
Level Four                AERO-2MJ      Engineering           N               Federal - Grants FE9         Total      0               25932.0
                                                                                                                      1                   0.0
                                                                                                                      2                   0.0

In [23]:
train, test = temporal_train_test_split(data, test_size=.2)

## Model

In [36]:
from sktime.forecasting.fbprophet import Prophet

In [37]:
Prophet?

Init signature:
Prophet(
    freq=None,
    add_seasonality=None,
    add_country_holidays=None,
    growth='linear',
    growth_floor=0.0,
    growth_cap=None,
    changepoints=None,
    n_changepoints=25,
    changepoint_range=0.8,
    yearly_seasonality='auto',
    weekly_seasonality='auto',
    daily_seasonality='auto',
    holidays=None,
    seasonality_mode='additive',
    seasonality_prior_scale=10.0,
    holidays_prior_scale=10.0,
    changepoint_prior_scale=0.05,
    mcmc_samples=0,
    alpha=0.05,
    uncertainty_samples=1000,
    stan_backend=None,
    verbose=0,
    fit_kwargs=None,
)
Docstring:     
Prophet forecaster by wrapping Facebook's prophet algorithm [1]_.

Direct interface to Facebook prophet, using the sktime interface.
All hyper-parameters are exposed via the constructor.

Data can be passed in one of the sktime compatible formats,
naming a column ``ds`` such as in the prophet package is not necessary.

Unlike vanilla ``prophet``, also supports integer/range and

In [38]:
from sktime.forecasting.fbprophet import Prophet
forecaster = Prophet()

In [39]:
prediction_length = 6
fh = [i for i in range(1, prediction_length+1)]
if forecaster.get_tag("requires-fh-in-fit"):
    forecaster.fit(train, fh=fh)
else: forecaster.fit(train)

17:06:28 - cmdstanpy - INFO - Chain [1] start processing
17:06:28 - cmdstanpy - INFO - Chain [1] done processing
17:06:28 - cmdstanpy - INFO - Chain [1] start processing
17:06:28 - cmdstanpy - INFO - Chain [1] done processing
17:06:28 - cmdstanpy - INFO - Chain [1] start processing
17:06:29 - cmdstanpy - INFO - Chain [1] done processing
17:06:29 - cmdstanpy - INFO - Chain [1] start processing
17:06:29 - cmdstanpy - INFO - Chain [1] done processing
17:06:29 - cmdstanpy - INFO - Chain [1] start processing
17:06:29 - cmdstanpy - INFO - Chain [1] done processing
17:06:29 - cmdstanpy - INFO - Chain [1] start processing
17:06:29 - cmdstanpy - INFO - Chain [1] done processing
17:06:29 - cmdstanpy - INFO - Chain [1] start processing
17:06:29 - cmdstanpy - INFO - Chain [1] done processing
17:06:29 - cmdstanpy - INFO - Chain [1] start processing
17:06:29 - cmdstanpy - INFO - Chain [1] done processing
17:06:29 - cmdstanpy - INFO - Chain [1] start processing
17:06:29 - cmdstanpy - INFO - Chain [1]

## Forecast

### Predict

In [40]:
y_pred = forecaster.predict(fh=fh)
y_pred

OFFER_BALANCE
ACADEMIC_LEVEL_TERM_START ACADEMIC_PLAN ACADEMIC_PROGRAM_DESC FIN_AID_FED_RES UVA_ACCESS       REPORT_CODE Need based AID_YEAR               
Level Four                AERO-2MJ      Engineering           N               Federal - Grants FE9         Total      8          -2482.362908
                                                                                                                      9          -3977.676115
                                                                                                                      10         -5472.989323
                                                                                                                      11         -6968.302530
                                                                                                                      12         -8463.615737
                                                                                                                      13         -9958.928944
                                                                                                           Y          8          -2482.362908
                                                                                                                      9          -3977.676115
                                                                                                                      10         -5472.989323
                                                                                                                      11         -6968.302530
                                                                                                                      12         -8463.615737
                                                                                                                      13         -9958.928944
                                                                                               Total       Total      8            746.767510
                                                                                                                      9           -208.356208
                                                                                                                      10         -1163.479927
                                                                                                                      11         -2118.603645
                                                                                                                      12         -3073.727364
                                                                                                                      13         -4028.851082
                                                                                                           Y          8            746.767510
                                                                                                                      9           -208.356208
                                                                                                                      10         -1163.479927
                                                                                                                      11         -2118.603645
                                                                                                                      12         -3073.727364
                                                                                                                      13         -4028.851082
                                                                              Outside - Grants OT3         N          8         -10893.090853
                                                                                                                      9         -14652.699617
                                                                                                                      10        -18412.308380
          

### Coverage

In [41]:
y_coverage = forecaster.predict_interval(fh=fh, coverage=0.9)
y_coverage

OFFER_BALANCE  \
                                                                                                                                         0.9   
                                                                                                                                       lower   
ACADEMIC_LEVEL_TERM_START ACADEMIC_PLAN ACADEMIC_PROGRAM_DESC FIN_AID_FED_RES UVA_ACCESS       REPORT_CODE Need based AID_YEAR                 
Level Four                AERO-2MJ      Engineering           N               Federal - Grants FE9         Total      8        -14593.518013   
                                                                                                                      9        -16297.418013   
                                                                                                                      10       -17846.896307   
                                                                                                                      11       -19250.550092   
                                                                                                                      12       -20188.317603   
                                                                                                                      13       -22729.788505   
                                                                                                           Y          8        -15531.321600   
                                                                                                                      9        -16399.735903   
                                                                                                                      10       -18971.495543   
                                                                                                                      11       -18982.068837   
                                                                                                                      12       -20713.896451   
                                                                                                                      13       -22359.909971   
                                                                                               Total       Total      8        -13606.260753   
                                                                                                                      9        -13836.454165   
                                                                                                                      10       -15698.043778   
                                                                                                                      11       -16310.889686   
                                                                                                                      12       -17500.639289   
                                                                                                                      13       -18268.949374   
                                                                                                           Y          8        -13232.228477   
                                                                                                                      9        -15084.928241   
                                                                                                                      10       -15397.000689   
                                                                                                                      11       -15737.548687   
                                                                                                                      12       -17327.107104   
                                                                                                                      13       -16900.335552   
                                                                              Outside - Grants 

### Quantiles

In [42]:
y_quantiles = forecaster.predict_quantiles(fh=fh)
y_quantiles

OFFER_BALANCE  \
                                                                                                                                        0.05   
ACADEMIC_LEVEL_TERM_START ACADEMIC_PLAN ACADEMIC_PROGRAM_DESC FIN_AID_FED_RES UVA_ACCESS       REPORT_CODE Need based AID_YEAR                 
Level Four                AERO-2MJ      Engineering           N               Federal - Grants FE9         Total      8        -15527.371674   
                                                                                                                      9        -17612.670029   
                                                                                                                      10       -18311.758785   
                                                                                                                      11       -18774.289095   
                                                                                                                      12       -21408.549021   
                                                                                                                      13       -23014.365049   
                                                                                                           Y          8        -15777.215556   
                                                                                                                      9        -16509.854472   
                                                                                                                      10       -18682.145472   
                                                                                                                      11       -19575.928193   
                                                                                                                      12       -20619.937697   
                                                                                                                      13       -22586.042068   
                                                                                               Total       Total      8        -13882.338653   
                                                                                                                      9        -14124.054070   
                                                                                                                      10       -15666.165029   
                                                                                                                      11       -16653.347451   
                                                                                                                      12       -17657.983418   
                                                                                                                      13       -18221.470849   
                                                                                                           Y          8        -13401.753554   
                                                                                                                      9        -13804.034996   
                                                                                                                      10       -14998.725227   
                                                                                                                      11       -15912.206635   
                                                                                                                      12       -17562.227888   
                                                                                                                      13       -18760.667407   
                                                                              Outside - Grants OT3         N          8        -28308.332398   
                                                                                               

## Update